In [41]:
#This is a starting point
#uncomment below to install merlin library
#!pip install merlin, nvtabular, merlin.models
#I had to downgrade keras to 12.2.0 there maybe issues regarding tensorflow in the future
#run lines below if you have also have an error ab not finding a keras package

#!pip uninstall keras
#!pip install keras==2.12.0


#These are the same imports from
#https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb

import os
import pandas as pd
import nvtabular as nvt
import matplotlib.pyplot as plt

from merlin.models.utils.example_utils import workflow_fit_transform
import merlin.io

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from nvtabular.ops import *
from merlin.core.utils import download_file
from merlin.schema.tags import Tags

# Load and split

In [71]:
#num rows
data_size = df.shape[0]

#We're doing a 20/80 train/validate split change the .2 to change the split
train_split_ratio = int(.2 * data_size)

train_data = df[train_split_ratio:].to_parquet("train_data.parquet")
valid_data = df[:train_split_ratio].to_parquet("valid_data.parquet")

train = Dataset("train_data.parquet")
valid = Dataset("valid_data.parquet")

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (1648670318) (bytes) of parquet file is bigger than requested part_size (1073741824) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(


# EDA

In [43]:
df = pd.read_csv('/Users/andrew/Desktop/projects/recsys_data/2023-10-05 9_23pm.csv') 
#  ^ This needs point to your local .csv too big for github

In [46]:
#accidentally added an extra CLICKSTREAM_EVENTS_TOTAL column
df = df.drop(columns = ['CLICKSTREAM_EVENTS_TOTAL.1'])

# NVTabular

In [55]:
#This is how to iniate a dataset using NVtabular an NVIDIA 
#optimized ETL/feature engineering(i think?) library
#data = nvt.io.dataset.Dataset('data/reviews.csv')

In [54]:
# df.columns

In [48]:
categorical_user_features = ['COUNTRY', 'DERIVED_GENDER_BY_NAME', 'EVENT_NAME']
categorical_item_features = ['STYLE', 'TAXONOMY_STYLE', 'COLOR_NAME', 'PRODUCT_CLASS', 'PRODUCT_SUBCLASS', 'TEAM', 'FRANCHISE', 'PRODUCT_GROUP']

In [ ]:
#Features needed to Transform still: FIRST_PURCHASE_AT, FIRST_VISIT_AT, LATEST_VISIT_AT
# LATEST_PURCHASE_AT, EVENT_TIMESTAMP

In [52]:
user_id = ["USER_ID"] >> Categorify(dtype = "int32") >> TagAsUserID()
item_id = ["ITEM_ID"] >> Categorify(dtype = "int32") >> TagAsItemID()
item_features = categorical_item_features >> Categorify(dtype = "int32") >> TagAsItemFeatures()
user_features = categorical_user_features >> Categorify(dtype = "int32") >> TagAsUserFeatures()

In [65]:
user_features

<Node TagAsUserFeatures output>

In [53]:
#Make this more succinct
CLICKSTREAM_EVENTS_TOTAL = (
    ['CLICKSTREAM_EVENTS_TOTAL']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsUserFeatures()
)

#has the price changed over time?
PRICE_INFORMATION = (
    ['PRICE_INFORMATION']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsItemFeatures()
)

AVG_REVIEW_SCORE = (
    ['AVG_REVIEW_SCORE']
    >>FillMissing(0)
    >>LogOp()
    >>Normalize()
    >>LambdaOp(lambda col: col.astype("float32"))
    >>TagAsItemFeatures()
)

In [68]:
outputs = (
    user_id + 
    user_features + 
    CLICKSTREAM_EVENTS_TOTAL + 
    item_id +
    item_features + 
    PRICE_INFORMATION +
    AVG_REVIEW_SCORE
)

workflow = nvt.Workflow(outputs)

In [73]:
%%time

train_data = Dataset('train_data.parquet') #Direct to your local files
valid_data = Dataset('valid_data.parquet')

workflow.fit(train_data)

workflow.transform(train_data).to_parquet(
    'train_data_transformed.parquet' #Feel free to replace with your own local file path
)
workflow.transform(valid_data).to_parquet(
    'valid_data_transformed.parquet' #same here
)

/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/parquet.py:343: UserWarning: Row group memory size (1648670318) (bytes) of parquet file is bigger than requested part_size (1073741824) for the NVTabular dataset.A row group memory size of 128 MB is generally recommended. You can find info on how to set the row group size of parquet files in https://nvidia-merlin.github.io/NVTabular/main/resources/troubleshooting.html#setting-the-row-group-size-for-the-parquet-files
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/merlin/io/dataset.py:267: UserWarning: Initializing an NVTabular Dataset in CPU mode.This is an experimental feature with extremely limited support!
  warnings.warn(
/Users/andrew/anaconda3/lib/python3.11/site-packages/pandas/co

CPU times: user 20.9 s, sys: 3.96 s, total: 24.8 s
Wall time: 25 s


/Users/andrew/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:402: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Transform Timestamped data

In [40]:
train.schema

,name,tags,dtype,is_list,is_ragged
0,USER_ID,(),"DType(name='object', element_type=<ElementType...",False,False
1,COUNTRY,(),"DType(name='object', element_type=<ElementType...",False,False
2,GENDER_MALE,(),"DType(name='int64', element_type=<ElementType....",False,False
3,GENDER_FEMALE,(),"DType(name='int64', element_type=<ElementType....",False,False
4,LOG_CLICKSTREAM_EVENTS_TOTAL,(),"DType(name='float64', element_type=<ElementTyp...",False,False
5,CLICKSTREAM_EVENTS_TOTAL,(),"DType(name='int64', element_type=<ElementType....",False,False
6,EVENT_NAME,(),"DType(name='object', element_type=<ElementType...",False,False
7,ITEM_ID,(),"DType(name='object', element_type=<ElementType...",False,False
8,PRICE_INFORMATION,(),"DType(name='float64', element_type=<ElementTyp...",False,False
9,AVG_REVIEW_SCORE,(),"DType(name='float64', element_type=<ElementTyp...",False,False


# Model

In [19]:
model = mm.DLRMModel(
    train.schema,                                                   # 1
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),                            # 2
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(train.schema)      # 3
)

model.compile(optimizer="adagrad", run_eagerly=False)
model.fit(train, validation_data=valid, batch_size=1024)
eval_metrics = model.evaluate(valid, batch_size=1024, return_dict=True)

ValueError: ('Binary classification task requires a column with a ', '`Tags.BINARY_CLASSIFICATION` tag.')